## Explore data

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## STEP 1 — Build the Local Database (load all files)

In [ ]:
import os
import json

def load_documents(root_dir):
    docs = []
    for path, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".txt"):
                fp = os.path.join(path, file)
                with open(fp, "r", encoding="utf-8") as f:
                    text = f.read()

                docs.append({
                    "path": fp,
                    "text": text
                })
    return docs


DATABASE_PATH = "/content/drive/MyDrive/MEDICAL_AGENT/RAGhealthcare/dermato"
docs = load_documents(DATABASE_PATH)
print("Loaded:", len(docs), "documents")


Loaded: 205 documents


## STEP 2 — Save keyword database (JSON)

In [ ]:
KEYWORD_DB_PATH = "/content/dermato_keyword_db.json"

with open(KEYWORD_DB_PATH, "w", encoding="utf-8") as f:
    json.dump(docs, f, ensure_ascii=False, indent=2)

print("Keyword  saved to", KEYWORD_DB_PATH)

Keyword  saved to /content/dermato_keyword_db.json


In [ ]:
def keyword_search(query, max_results=5):
    query = query.lower()
    results = []
    for d in docs:
        if query in d["text"].lower() or query in d["path"].lower():
            results.append(d)
    return results[:max_results]


## STEP 3 — Build & Save Chroma Vector Database

In [ ]:
!pip install chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.7 MB/s 

In [ ]:
# ===============================================
# CELL 1 — BUILD SEMANTIC SEARCH DATABASE
# ===============================================

import os
import json
import shutil
import chromadb
from sentence_transformers import SentenceTransformer

# -------------------------------------------------
# 1) Load keyword database (already created)
# -------------------------------------------------
KEYWORD_DB_PATH = "/content/dermato_keyword_db.json"

with open(KEYWORD_DB_PATH, "r", encoding="utf-8") as f:
    docs = json.load(f)

print("Loaded docs:", len(docs))

# -------------------------------------------------
# 2) Create persistent Chroma DB ou il ya les vecteurs
# Chroma vecter database
# -------------------------------------------------
DB_DIR = "/content/dermato_vector_db"
client = chromadb.PersistentClient(path=DB_DIR)
collection = client.get_or_create_collection(
    name="dermato",
    metadata={"hnsw:space": "cosine"}
)
print("Persistent Chroma collection created.")

# -------------------------------------------------
# 4) Load embedding model
# -------------------------------------------------
model = SentenceTransformer("all-MiniLM-L6-v2")
print("Embedding model loaded.")

# -------------------------------------------------
# 5) Insert documents into Chroma in batches
# -------------------------------------------------
batch_size = 40
batch_docs, batch_ids, batch_metas = [], [], []

for i, d in enumerate(docs):
    batch_ids.append(str(i))
    batch_docs.append(d["text"])
    batch_metas.append({"path": d["path"]})

    # Insert after every batch
    if len(batch_docs) == batch_size or i == len(docs) - 1:
        embeddings = model.encode(batch_docs).tolist()

        collection.add(
            ids=batch_ids,
            embeddings=embeddings,
            documents=batch_docs,
            metadatas=batch_metas
        )

        print(f"Inserted batch ending at index {i}")
        batch_docs, batch_ids, batch_metas = [], [], []

print("Total documents stored in vector DB:", collection.count())

# -------------------------------------------------
# 6) Define semantic search function
# -------------------------------------------------
def semantic_search(query, top_k=5):
    query_emb = model.encode(query).tolist()

    results = collection.query(
        query_embeddings=[query_emb],
        n_results=top_k,
        include=["distances", "metadatas", "documents"]                        #  obligatoire pour obtenir les scores
    )

    output = []
    documents = results["documents"][0]
    metadatas = results["metadatas"][0]
    distances = results["distances"][0]

    for doc, meta, dist in zip(documents, metadatas, distances):
        output.append({
            "path": meta["path"],
            "preview": doc[:500],
            "score": dist                                                       #  voici le score !
        })

    return output


print("Semantic search system ready.")


Loaded docs: 205
Persistent Chroma collection created.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded.
Inserted batch ending at index 39
Inserted batch ending at index 79
Inserted batch ending at index 119
Inserted batch ending at index 159
Inserted batch ending at index 199
Inserted batch ending at index 204
Total documents stored in vector DB: 205
Semantic search system ready.


Hybrid Search Test Function

Test it

In [ ]:
def hybrid_search(query):
    return {
        "keyword_results": keyword_search(query),
        "semantic_results": semantic_search(query)
    }

In [ ]:
result = hybrid_search("melanoma causes")

print("=== KEYWORD RESULTS ===")
for r in result["keyword_results"]:
    print(r["path"])

print("\n=== SEMANTIC RESULTS ===")
for r in result["semantic_results"]:
    print(f"{r['path']}   --> score: {r['score']}")


=== KEYWORD RESULTS ===

=== SEMANTIC RESULTS ===
/content/drive/MyDrive/MEDICAL_AGENT/RAGhealthcare/dermato/datadermatocancer/patient_cases_and_clinical_patterns/case_melanoma_early/causes.txt   --> score: 0.31000179052352905
/content/drive/MyDrive/MEDICAL_AGENT/RAGhealthcare/dermato/datadermatocancer/what is cancer/Which cancers cause the most deaths/woaman/Mélanome MalinF.txt   --> score: 0.32546746730804443
/content/drive/MyDrive/MEDICAL_AGENT/RAGhealthcare/dermato/datadermatocancer/what is cancer/Which cancers cause the most deaths/men/Mélanome MalinH.txt   --> score: 0.3287009000778198
/content/drive/MyDrive/MEDICAL_AGENT/RAGhealthcare/dermato/datadermatocancer/types of cancer/ocular melanoma/Causes.txt   --> score: 0.33485209941864014
/content/drive/MyDrive/MEDICAL_AGENT/RAGhealthcare/dermato/datadermatocancer/types of cancer/melanoma/Causes.txt   --> score: 0.3502214550971985


In [ ]:
result["semantic_results"][0]

{'path': '/content/drive/MyDrive/MEDICAL_AGENT/RAGhealthcare/dermato/datadermatocancer/patient_cases_and_clinical_patterns/case_melanoma_early/causes.txt',
 'preview': 'What is the cause of superficial spreading melanoma?\nSuperficial spreading melanoma is due to the development\n\n of malignant pigment cells (melanocytes) along the basal layer of the epidermis. The majority arise in previously normal-appearing skin. About 25% develop within an existing melanocytic naevus, which can be a normal common naevus, an atypical or dysplastic naevus, or a congenital naevus.\n\nWhat triggers the melanocytes to become malignant is not fully known. Specific gene\n\n mutations su',
 'score': 0.31000179052352905}

# Step 4-  Evaluation

## Hybrid

In [ ]:
import os
import time
import math

# ==========================================================
#    GLOBAL RETRIEVAL EVALUATION: RECALL@K, PRECISION@K,
#    MRR, nDCG@K
# ==========================================================

def evaluate_entire_database(docs_list, top_k=5):
    total_docs = len(docs_list)
    if total_docs == 0:
        print("❌ No documents to evaluate")
        return

    found_count = 0          # For Recall@K
    precision_sum = 0        # For Precision@K
    mrr_sum = 0              # For Mean Reciprocal Rank
    ndcg_sum = 0             # For nDCG@K
    total_time = 0

    print(f"\n🚀 STARTING GLOBAL EVALUATION ON {total_docs} DOCUMENTS...")
    print("="*75)
    print(f"{'QUERY SOURCE':<50} | {'STATUS':<10} | {'RANK':<5}")
    print("-" * 75)

    for doc in docs_list:
        target_path = doc["path"]

        # -------------------------------------------------
        # 1. BUILD QUERY FROM PATH
        # -------------------------------------------------
        parts = target_path.split("/")
        parent_folder = parts[-2] if len(parts) >= 2 else ""
        filename = os.path.splitext(os.path.basename(target_path))[0]

        folder_clean = parent_folder.replace("_", " ")
        filename_clean = filename.replace("_", " ")
        generated_query = f"{folder_clean} {filename_clean}"

        # -------------------------------------------------
        # 2. RUN HYBRID SEARCH
        # -------------------------------------------------
        start = time.time()
        search_res = hybrid_search(generated_query)
        end = time.time()
        total_time += (end - start)

        # Collect results
        retrieved_paths = []
        semantic = search_res.get("semantic_results") or []
        keyword = search_res.get("keyword_results") or []

        for r in semantic:
            retrieved_paths.append(r["path"])
        for r in keyword:
            if r["path"] not in retrieved_paths:
                retrieved_paths.append(r["path"])

        retrieved_paths = retrieved_paths[:top_k]

        # -------------------------------------------------
        # 3. METRIC CALCULATIONS
        # -------------------------------------------------

        # REC@K
        found = target_path in retrieved_paths
        if found:
            found_count += 1
            rank = retrieved_paths.index(target_path) + 1
            print(f"{generated_query[:48]:<50} | {'FOUND':<10} | #{rank}")

            # PREC@K (each doc has only 1 relevant answer)
            precision_sum += 1 / top_k

            # MRR
            mrr_sum += 1 / rank

            # nDCG@K = 1 / log2(rank+1)
            ndcg_sum += 1 / math.log2(rank + 1)

        else:
            print(f"{generated_query[:48]:<50} | {'MISSED':<10} | -")

            # Precision contribution is 0 if missed
            # MRR contribution is 0
            # nDCG contribution is 0

    # ==========================================================
    # FINAL METRICS
    # ==========================================================
    recall_at_k = found_count / total_docs
    precision_at_k = precision_sum / total_docs
    mrr = mrr_sum / total_docs
    ndcg_at_k = ndcg_sum / total_docs
    avg_latency = total_time / total_docs

    print("\n" + "="*60)
    print("📊 GLOBAL RETRIEVAL METRICS")
    print("="*60)
    print(f"Total Documents Evaluated: {total_docs}")
    print(f"Recall@{top_k}:           {recall_at_k*100:.2f}%")
    print(f"Precision@{top_k}:        {precision_at_k*100:.2f}%")
    print(f"MRR:                      {mrr:.4f}")
    print(f"nDCG@{top_k}:             {ndcg_at_k:.4f}")
    print(f"Avg Query Latency:        {avg_latency:.4f} sec")
    print("="*60 + "\n")



In [ ]:
# Run the evaluation
evaluate_entire_database(docs, top_k=5)


🚀 STARTING GLOBAL EVALUATION ON 205 DOCUMENTS...
QUERY SOURCE                                       | STATUS     | RANK 
---------------------------------------------------------------------------
Cancer side effects chemotherapy skin reactions.   | FOUND      | #4
Cancer side effects Fatigue                        | FOUND      | #1
Cancer side effects hair loss                      | FOUND      | #1
Cancer side effects radiation dermatitis           | MISSED     | -
Cancer side effects targeted therapy               | FOUND      | #2
Cancer side effects psychological effects          | FOUND      | #2
Cancer side effects immunotherapy skin             | FOUND      | #1
Cancer side effects Lymphoedema                    | FOUND      | #3
Cancer side effects Peripheral neuropathy          | FOUND      | #1
causes causes carcinome basocellulaire             | MISSED     | -
causes causes carcinome merkel                     | FOUND      | #2
causes causes melanoma                       

## Key word search

In [ ]:
def hybrid_search(query):
    return {
        "keyword_results": keyword_search(query),
        #"semantic_results": semantic_search(query)
    }

In [ ]:
import os
import time
import math

# ==========================================================
#    GLOBAL RETRIEVAL EVALUATION: RECALL@K, PRECISION@K,
#    MRR, nDCG@K
# ==========================================================

def evaluate_entire_database(docs_list, top_k=5):
    total_docs = len(docs_list)
    if total_docs == 0:
        print("❌ No documents to evaluate")
        return

    found_count = 0          # For Recall@K
    precision_sum = 0        # For Precision@K
    mrr_sum = 0              # For Mean Reciprocal Rank
    ndcg_sum = 0             # For nDCG@K
    total_time = 0

    print(f"\n🚀 STARTING GLOBAL EVALUATION ON {total_docs} DOCUMENTS...")
    print("="*75)
    print(f"{'QUERY SOURCE':<50} | {'STATUS':<10} | {'RANK':<5}")
    print("-" * 75)

    for doc in docs_list:
        target_path = doc["path"]

        # -------------------------------------------------
        # 1. BUILD QUERY FROM PATH
        # -------------------------------------------------
        parts = target_path.split("/")
        parent_folder = parts[-2] if len(parts) >= 2 else ""
        filename = os.path.splitext(os.path.basename(target_path))[0]

        folder_clean = parent_folder.replace("_", " ")
        filename_clean = filename.replace("_", " ")
        generated_query = f"{folder_clean} {filename_clean}"

        # -------------------------------------------------
        # 2. RUN HYBRID SEARCH
        # -------------------------------------------------
        start = time.time()
        search_res = hybrid_search(generated_query)
        end = time.time()
        total_time += (end - start)

        # Collect results
        retrieved_paths = []
        semantic = search_res.get("semantic_results") or []
        keyword = search_res.get("keyword_results") or []

        for r in semantic:
            retrieved_paths.append(r["path"])
        for r in keyword:
            if r["path"] not in retrieved_paths:
                retrieved_paths.append(r["path"])

        retrieved_paths = retrieved_paths[:top_k]

        # -------------------------------------------------
        # 3. METRIC CALCULATIONS
        # -------------------------------------------------

        # REC@K
        found = target_path in retrieved_paths
        if found:
            found_count += 1
            rank = retrieved_paths.index(target_path) + 1
            print(f"{generated_query[:48]:<50} | {'FOUND':<10} | #{rank}")

            # PREC@K (each doc has only 1 relevant answer)
            precision_sum += 1 / top_k

            # MRR
            mrr_sum += 1 / rank

            # nDCG@K = 1 / log2(rank+1)
            ndcg_sum += 1 / math.log2(rank + 1)

        else:
            print(f"{generated_query[:48]:<50} | {'MISSED':<10} | -")

            # Precision contribution is 0 if missed
            # MRR contribution is 0
            # nDCG contribution is 0

    # ==========================================================
    # FINAL METRICS
    # ==========================================================
    recall_at_k = found_count / total_docs
    precision_at_k = precision_sum / total_docs
    mrr = mrr_sum / total_docs
    ndcg_at_k = ndcg_sum / total_docs
    avg_latency = total_time / total_docs

    print("\n" + "="*60)
    print("📊 GLOBAL RETRIEVAL METRICS")
    print("="*60)
    print(f"Total Documents Evaluated: {total_docs}")
    print(f"Recall@{top_k}:           {recall_at_k*100:.2f}%")
    print(f"Precision@{top_k}:        {precision_at_k*100:.2f}%")
    print(f"MRR:                      {mrr:.4f}")
    print(f"nDCG@{top_k}:             {ndcg_at_k:.4f}")
    print(f"Avg Query Latency:        {avg_latency:.4f} sec")
    print("="*60 + "\n")



In [ ]:
# Run the evaluation
evaluate_entire_database(docs, top_k=5)


🚀 STARTING GLOBAL EVALUATION ON 205 DOCUMENTS...
QUERY SOURCE                                       | STATUS     | RANK 
---------------------------------------------------------------------------
Cancer side effects chemotherapy skin reactions.   | MISSED     | -
Cancer side effects Fatigue                        | MISSED     | -
Cancer side effects hair loss                      | MISSED     | -
Cancer side effects radiation dermatitis           | MISSED     | -
Cancer side effects targeted therapy               | MISSED     | -
Cancer side effects psychological effects          | MISSED     | -
Cancer side effects immunotherapy skin             | MISSED     | -
Cancer side effects Lymphoedema                    | MISSED     | -
Cancer side effects Peripheral neuropathy          | MISSED     | -
causes causes carcinome basocellulaire             | MISSED     | -
causes causes carcinome merkel                     | MISSED     | -
causes causes melanoma                             | M

## semantic search

In [ ]:
def hybrid_search(query):
    return {
        #"keyword_results": keyword_search(query),
        "semantic_results": semantic_search(query)
    }

In [ ]:
import os
import time
import math

# ==========================================================
#    GLOBAL RETRIEVAL EVALUATION: RECALL@K, PRECISION@K,
#    MRR, nDCG@K
# ==========================================================

def evaluate_entire_database(docs_list, top_k=5):
    total_docs = len(docs_list)
    if total_docs == 0:
        print("❌ No documents to evaluate")
        return

    found_count = 0          # For Recall@K
    precision_sum = 0        # For Precision@K
    mrr_sum = 0              # For Mean Reciprocal Rank
    ndcg_sum = 0             # For nDCG@K
    total_time = 0

    print(f"\n🚀 STARTING GLOBAL EVALUATION ON {total_docs} DOCUMENTS...")
    print("="*75)
    print(f"{'QUERY SOURCE':<50} | {'STATUS':<10} | {'RANK':<5}")
    print("-" * 75)

    for doc in docs_list:
        target_path = doc["path"]

        # -------------------------------------------------
        # 1. BUILD QUERY FROM PATH
        # -------------------------------------------------
        parts = target_path.split("/")
        parent_folder = parts[-2] if len(parts) >= 2 else ""
        filename = os.path.splitext(os.path.basename(target_path))[0]

        folder_clean = parent_folder.replace("_", " ")
        filename_clean = filename.replace("_", " ")
        generated_query = f"{folder_clean} {filename_clean}"

        # -------------------------------------------------
        # 2. RUN HYBRID SEARCH
        # -------------------------------------------------
        start = time.time()
        search_res = hybrid_search(generated_query)
        end = time.time()
        total_time += (end - start)

        # Collect results
        retrieved_paths = []
        semantic = search_res.get("semantic_results") or []
        keyword = search_res.get("keyword_results") or []

        for r in semantic:
            retrieved_paths.append(r["path"])
        for r in keyword:
            if r["path"] not in retrieved_paths:
                retrieved_paths.append(r["path"])

        retrieved_paths = retrieved_paths[:top_k]

        # -------------------------------------------------
        # 3. METRIC CALCULATIONS
        # -------------------------------------------------

        # REC@K
        found = target_path in retrieved_paths
        if found:
            found_count += 1
            rank = retrieved_paths.index(target_path) + 1
            print(f"{generated_query[:48]:<50} | {'FOUND':<10} | #{rank}")

            # PREC@K (each doc has only 1 relevant answer)
            precision_sum += 1 / top_k

            # MRR
            mrr_sum += 1 / rank

            # nDCG@K = 1 / log2(rank+1)
            ndcg_sum += 1 / math.log2(rank + 1)

        else:
            print(f"{generated_query[:48]:<50} | {'MISSED':<10} | -")

            # Precision contribution is 0 if missed
            # MRR contribution is 0
            # nDCG contribution is 0

    # ==========================================================
    # FINAL METRICS
    # ==========================================================
    recall_at_k = found_count / total_docs
    precision_at_k = precision_sum / total_docs
    mrr = mrr_sum / total_docs
    ndcg_at_k = ndcg_sum / total_docs
    avg_latency = total_time / total_docs

    print("\n" + "="*60)
    print("📊 GLOBAL RETRIEVAL METRICS")
    print("="*60)
    print(f"Total Documents Evaluated: {total_docs}")
    print(f"Recall@{top_k}:           {recall_at_k*100:.2f}%")
    print(f"Precision@{top_k}:        {precision_at_k*100:.2f}%")
    print(f"MRR:                      {mrr:.4f}")
    print(f"nDCG@{top_k}:             {ndcg_at_k:.4f}")
    print(f"Avg Query Latency:        {avg_latency:.4f} sec")
    print("="*60 + "\n")



In [ ]:
# Run the evaluation
evaluate_entire_database(docs, top_k=5)


🚀 STARTING GLOBAL EVALUATION ON 205 DOCUMENTS...
QUERY SOURCE                                       | STATUS     | RANK 
---------------------------------------------------------------------------
Cancer side effects chemotherapy skin reactions.   | FOUND      | #4
Cancer side effects Fatigue                        | FOUND      | #1
Cancer side effects hair loss                      | FOUND      | #1
Cancer side effects radiation dermatitis           | MISSED     | -
Cancer side effects targeted therapy               | FOUND      | #2
Cancer side effects psychological effects          | FOUND      | #2
Cancer side effects immunotherapy skin             | FOUND      | #1
Cancer side effects Lymphoedema                    | FOUND      | #3
Cancer side effects Peripheral neuropathy          | FOUND      | #1
causes causes carcinome basocellulaire             | MISSED     | -
causes causes carcinome merkel                     | FOUND      | #2
causes causes melanoma                       

📊 Interprétation des métriques globales de récupération

Recall@5 : 81.46%
→ Commentaire : Dans plus de 8 cas sur 10, le document médical pertinent est retrouvé parmi les cinq premiers résultats.
→ Interprétation : Le moteur RAG est capable de fournir des sources médicales pertinentes dans la majorité des situations, ce qui est essentiel pour alimenter correctement le raisonnement de l’agent IA et limiter les réponses incomplètes.

Precision@5 : 16.29%
→ Commentaire : En moyenne, un seul document pertinent est présent parmi les cinq documents retournés.
→ Interprétation : Ce résultat est attendu dans un contexte où chaque requête n’a qu’un seul document de référence pertinent. La priorité étant donnée au rappel plutôt qu’à la précision stricte, cette valeur reste acceptable pour un système d’aide à la décision médicale.

MRR (Mean Reciprocal Rank) : 0.5644
→ Commentaire : Le document pertinent apparaît généralement parmi les premières positions des résultats (souvent en rang 1 ou 2).
→ Interprétation : Le classement des documents est efficace, ce qui permet au modèle de langage d’accéder rapidement aux informations les plus importantes et d’améliorer la qualité du diagnostic raisonné.

nDCG@5 : 0.6265
→ Commentaire : Le score indique une bonne qualité globale de l’ordre des résultats retournés.
→ Interprétation : Les documents les plus pertinents sont correctement priorisés, confirmant la cohérence du moteur de recherche sémantique dans un contexte médical.

Latence moyenne : 0.0113 seconde
→ Commentaire : Le temps de réponse par requête est très faible.
→ Interprétation : Le moteur est suffisamment rapide pour une utilisation en temps réel, ce qui le rend parfaitement adapté à une intégration dans une application mobile médicale.
